# Modelagem de Sentimento de CX (PT-BR)

Este notebook documenta o processo de treinamento do modelo de análise de sentimento utilizado no projeto.

## Objetivos
1. Carregar datasets de redes sociais (Twitter) em PT-BR.
2. Pré-processar o texto (limpeza, normalização).
3. Treinar um modelo de Regressão Logística com TF-IDF.
4. Avaliar métricas de performance.
5. Exportar o modelo para produção.

In [ ]:
import pandas as pd
import numpy as np
import joblib
import re
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Carregamento de Dados

Vamos carregar todos os arquivos CSV disponíveis na pasta `data/TrainingDataSets`.

In [ ]:
def load_and_combine_data():
    # Ajuste o caminho se necessário (ex: '../data/TrainingDataSets' se rodar de dentro de 'notebooks/')
    path = '../data/TrainingDataSets'
    all_files = glob.glob(os.path.join(path, "*.csv"))
    
    df_list = []
    for filename in all_files:
        print(f"Carregando {filename}...")
        try:
            df = pd.read_csv(filename)
            
            # Normalização de colunas
            if 'tweet_text' in df.columns:
                df = df.rename(columns={'tweet_text': 'text'})
            elif 'review_text' in df.columns:
                df = df.rename(columns={'review_text': 'text'})
            
            if 'sentiment' in df.columns and 'text' in df.columns:
                df_list.append(df[['text', 'sentiment']])
        except Exception as e:
            print(f"Erro ao carregar {filename}: {e}")
            
    if not df_list:
        raise ValueError("Nenhum dado carregado!")
        
    return pd.concat(df_list, axis=0, ignore_index=True)

df = load_and_combine_data()
print(f"Total de amostras: {len(df)}")
print(df['sentiment'].value_counts())

## 2. Pré-processamento

Normalização dos rótulos para obter classes consistentes (Bom, Ruim, Neutro) e limpeza do texto.

In [ ]:
# Mapeamento de Rótulos
norm_map = {
    'Positivo': 'Bom', 'Positive': 'Bom', 'Bom': 'Bom',
    'Negativo': 'Ruim', 'Negative': 'Ruim', 'Ruim': 'Ruim',
    'Neutro': 'Neutro', 'Neutral': 'Neutro'
}

df['sentiment_label'] = df['sentiment'].map(norm_map)
df = df.dropna(subset=['sentiment_label'])

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'<[^>]+>', '', text) # HTML
    text = re.sub(r'http\S+|www\.\S+', '', text) # URLs
    text = re.sub(r'@\w+', '', text) # Handles
    text = re.sub(r'[^a-zA-Z\u00C0-\u00FF\s]', '', text) # Caracteres especiais
    return text.lower().strip()

df['clean_text'] = df['text'].apply(clean_text)
df = df[df['clean_text'].str.len() > 2]

print("Amostra após limpeza:")
print(df[['clean_text', 'sentiment_label']].head())

## 3. Treinamento do Modelo

Utilizaremos um Pipeline com:
- **TfidfVectorizer**: Para transformar texto em números (uni-gramas e bj-gramas).
- **LogisticRegression**: Modelo adequado para classificação textual com boa interpretabilidade.

In [ ]:
X = df['clean_text']
y = df['sentiment_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), min_df=5, max_features=100000)),
    ('clf', LogisticRegression(class_weight='balanced', max_iter=2000, n_jobs=-1))
])

print("Treinando...")
pipeline.fit(X_train, y_train)
print("Treinamento concluído.")

## 4. Avaliação

Verificação das métricas no conjunto de teste.

In [ ]:
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

## 5. Salvar Modelo

Persistência do pipeline para uso no Streamlit.

In [ ]:
# Salvar na pasta correta (subindo um nível)
output_path = '../data/sentiment_model.pkl'
joblib.dump(pipeline, output_path)
print(f"Modelo salvo em {output_path}")